# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests as r
import math
from scipy import stats
import xlsxwriter as xl

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN 

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = r.get(api_url).json()
#data.status_code

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.7974616358908527

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

### Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
df = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = r.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        row = pd.Series([symbol,
                         data[symbol]['quote']['latestPrice'],
                         data[symbol]['stats']['year1ChangePercent'],
                         'N/A'], index = my_columns)
        df = df.append(row, ignore_index = True)

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
df.sort_values('One-Year Price Return', ascending = False, inplace = True)
df = df[:50]
df.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your porfolio: ')
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again: ')
        portfolio_size = input('Enter the size of your porfolio: ')

portfolio_input()
print(portfolio_size)

Enter the size of your porfolio: 1000000
1000000


In [9]:
position_size = float(portfolio_size)/len(df.index)
df['Number of Shares to Buy'] = df['Price'].apply(lambda x: math.floor(position_size/x))

## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = ['Ticker', 'Price', 'Number of Shares to Buy', 'One Year Price Return', 'One Year Return Percentile', 'Six Month Price Return', 'Six Month Return Percentile', 'Three Month Price Return', 'Three Month Return Percentile', 'One Month Price Return', 'One Month Return Percentile', 'HQM Score']
hqm_df = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = r.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        row = pd.Series([symbol, 
                         data[symbol]['quote']['latestPrice'],'N/A',
                         data[symbol]['stats']['year1ChangePercent'], 'N/A',
                         data[symbol]['stats']['month6ChangePercent'], 'N/A',
                         data[symbol]['stats']['month3ChangePercent'], 'N/A',
                         data[symbol]['stats']['month1ChangePercent'], 'N/A',
                        'N/A'],
                        index = hqm_columns)
        hqm_df = hqm_df.append(row, ignore_index = True)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
hqm_df.head()
#stats.percentileofscore(list(hqm_df[f'{time_period} Price Return'].values), hqm_df.loc[1, f'{time_period} Price Return'])

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,A,124.15,N/A,0.435568,N/A,0.363443,N/A,0.166898,N/A,0.0420908,N/A,N/A
1,AAL,16.11,N/A,-0.455593,N/A,0.240159,N/A,0.183656,N/A,-0.0607183,N/A,N/A
2,AAP,173.10,N/A,-0.00665837,N/A,0.114223,N/A,0.0100164,N/A,0.0328954,N/A,N/A
3,AAPL,129.80,N/A,0.805581,N/A,0.449331,N/A,0.128501,N/A,0.0734628,N/A,N/A
4,ABBV,107.95,N/A,0.270519,N/A,0.109356,N/A,0.237931,N/A,-0.00734521,N/A,N/A


In [188]:
#There are None's in the columns and I can't get them and it causes an error.
time_periods = ['One Year', 'Six Month', 'Three Month', 'One Month']
for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_df.loc[hqm_df[change_col].isin([None]),[change_col]] = 0
        hqm_df.loc[row, percentile_col] = stats.percentileofscore(hqm_df[change_col], hqm_df.loc[row, f'{time_period} Price Return'])/100

In [189]:
hqm_df.head(10)

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,A,119.66,N/A,0.412545,0.871287,0.356008,0.689109,0.173696,0.516832,0.0280674,0.544554,N/A
1,AAL,15.96,N/A,-0.464666,0.019802,0.242142,0.481188,0.266224,0.69901,0.107694,0.89901,N/A
2,AAP,164.79,N/A,-0.00922334,0.352475,0.115075,0.269307,0.0188024,0.184158,0.0347997,0.59604,N/A
3,AAPL,133.78,N/A,0.852535,0.972277,0.470364,0.821782,0.142619,0.457426,0.0849284,0.835644,N/A
4,ABBV,111.87,N/A,0.288129,0.772277,0.112581,0.261386,0.256626,0.677228,0.0306794,0.566337,N/A
5,ABC,98.14,N/A,0.172362,0.621782,-0.00907183,0.0792079,0.0299193,0.215842,-0.044714,0.0613861,N/A
6,ABMD,327.40,N/A,0.925826,0.978218,0.27877,0.558416,0.245535,0.655446,0.215863,0.986139,N/A
7,ABT,111.25,N/A,0.282577,0.766337,0.205655,0.435644,0.0115269,0.178218,0.00958814,0.39604,N/A
8,ACN,270.03,N/A,0.266817,0.738614,0.231422,0.469307,0.166601,0.510891,0.0361631,0.607921,N/A
9,ADBE,516.07,N/A,0.533095,0.926733,0.13922,0.312871,0.00127058,0.144554,0.0440027,0.655446,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [192]:
# average retunr percentile across all time_period per row.
from statistics import mean
for row in hqm_df.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentile)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [197]:
hqm_df.sort_values('HQM Score', ascending = False, inplace = True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [199]:
portfolio_input()

Enter the size of your porfolio: 1000000


In [201]:
position_size = float(portfolio_size)/len(hqm_df.index)
hqm_df['Number of Shares to Buy'] = hqm_df['Price'].apply(lambda x: math.floor(position_size/x))
hqm_df

,Ticker,Price,Number of Shares to Buy,One Year Price Return,One Year Return Percentile,Six Month Price Return,Six Month Return Percentile,Three Month Price Return,Three Month Return Percentile,One Month Price Return,One Month Return Percentile,HQM Score
0,FCX,26.23,762,1.01024,0.988119,1.27472,0.99604,0.687289,0.978218,0.104662,0.889109,0.962871
1,ALB,151.62,131,1.0796,0.992079,0.96387,0.988119,0.733695,0.988119,0.0912389,0.863366,0.957921
2,SIVB,403.09,49,0.558155,0.932673,0.889623,0.976238,0.599684,0.970297,0.0960423,0.873267,0.938119
3,ALGN,548.34,36,0.93619,0.982178,0.965348,0.990099,0.713299,0.986139,0.0705939,0.788119,0.936634
4,MU,77.85,256,0.406209,0.859406,0.537087,0.889109,0.595306,0.968317,0.124806,0.928713,0.911386
5,DIS,186.94,106,0.265028,0.732673,0.613556,0.928713,0.480653,0.932673,0.221409,0.990099,0.89604
6,DVA,119.80,166,0.572119,0.934653,0.471226,0.825743,0.431052,0.90495,0.0841846,0.827723,0.873267
7,TPR,31.10,643,0.169722,0.615842,1.43218,0.99802,0.983966,0.99604,0.0968197,0.877228,0.871782
8,PAYC,465.23,42,0.741576,0.956436,0.423493,0.770297,0.430977,0.90297,0.0897831,0.855446,0.871287
9,ALXN,160.71,124,0.447905,0.891089,0.397239,0.740594,0.365967,0.843564,0.290265,0.99802,0.868317


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [204]:
hqm_df.to_csv('C:\\Users\\scott\\PycharmProjects\\quantitative_finance\\algorithmic_trading_youtube\\starter_files\\recommended trades2.csv', index = False)